In [5]:
from pytrends.request import TrendReq
import pandas as pd
import time

# Note: Due to pytrend's request limitations, we utilized a VPN to retrieve the complete dataset. 
# Additionally, data acquisition was more feasible from certain countries, such as Canada, compared 
# to others like Switzerland, where we frequently encountered issues with receiving empty arrays.
# Initializing pytrends...
pytrend = TrendReq()

#Category names that we are interested in , together with their IDS in google trends data
#Example 
category_names= [   
    "Energy & Utilities"
]
category_ids= [233]



# List of countries that are relevant for our project
countries = [
    "US",  # United States
    "CN",  # China
    "JP",  # Japan
    "DE",  # Germany
    "KR",  # South Korea
    "FR",  # France
    "GB",  # United Kingdom
    "CA",  # Canada
    "CH",  # Switzerland
   
]

# Set timeframe
timeframe = '2005-01-01 2020-12-31'

# Dictionary to hold data frames for each country
country_data_frames = {}

# Loop over each country
for country in countries:
    
    print(f"Fetching Data for {country}")
    # Initialize an empty list to collect data frames for this country
    dfs = []
    # Fetch the data for each category and add to the list
    for category_id in category_ids:
        attempts = 0
        success = False
        while not success and attempts < 5 :  # Retry up to 5 times
            try:
                pytrend.build_payload(kw_list=[" "], cat=category_id, timeframe=timeframe, geo=country)
                data = pytrend.interest_over_time()    
                #This printing was very useful, since it allows us to see when google trends returns an empty array,
                #sometimes even when the data could be fetched.
                print(data)
                # Drop the 'isPartial' column
                if 'isPartial' in data.columns:
                    data = data.drop(columns=['isPartial'])
                dfs.append(data)
                success = True
            except Exception as e:
                print(e)
                time.sleep(5)  # Sleep for 5 seconds
                attempts += 1

    # Concatenate all data frames along the columns
    combined_data = pd.concat(dfs, axis=1)

    # Store the combined data frame in the dictionary
    country_data_frames[country] = combined_data



Fetching Data for US
                isPartial
date                     
2005-01-01   0      False
2005-02-01   0      False
2005-03-01   0      False
2005-04-01   0      False
2005-05-01   0      False
...         ..        ...
2020-08-01  86      False
2020-09-01  80      False
2020-10-01  67      False
2020-11-01  68      False
2020-12-01  72      False

[192 rows x 2 columns]
Fetching Data for CN
               isPartial
date                    
2005-01-01  0      False
2005-02-01  0      False
2005-03-01  0      False
2005-04-01  0      False
2005-05-01  0      False
...        ..        ...
2020-08-01  5      False
2020-09-01  5      False
2020-10-01  5      False
2020-11-01  6      False
2020-12-01  6      False

[192 rows x 2 columns]
Fetching Data for JP
                isPartial
date                     
2005-01-01  23      False
2005-02-01  22      False
2005-03-01  21      False
2005-04-01  21      False
2005-05-01  22      False
...         ..        ...
2020-08-01   0    

In [6]:
# Create a list to hold all DataFrames for all countries
all_dataframes = []

# Loop over each country's DataFrame in country_data_frames
for country, df in country_data_frames.items():
    
    #Naming the columns of the dataframe
    df.columns=category_names
    # Add a new column with the country name
    df['Country'] = country  # This adds the country code as a new column

    # Append the DataFrame to the list
    all_dataframes.append(df)

# Concatenate all DataFrames in the list
combined_data = pd.concat(all_dataframes)

# Reset the index of the combined DataFrame
combined_data.reset_index(inplace=True)

#Add the year and month columns
combined_data["year"] = combined_data["date"].squeeze().apply(lambda x : int(str(x).split("-")[0]))
combined_data["month"] = combined_data["date"].squeeze().apply(lambda x : int(str(x).split("-")[1]))

combined_data

,date,Energy & Utilities,Country,year,month
0,2005-01-01,0,US,2005,1
1,2005-02-01,0,US,2005,2
2,2005-03-01,0,US,2005,3
3,2005-04-01,0,US,2005,4
4,2005-05-01,0,US,2005,5
...,...,...,...,...,...
1723,2020-08-01,49,CH,2020,8
1724,2020-09-01,49,CH,2020,9
1725,2020-10-01,47,CH,2020,10
1726,2020-11-01,48,CH,2020,11


In [ ]:
combined_data.to_csv("toy_example",index=False)